# 🔍 Bubble Wallet Clustering Analysis

**Objective**: Identify suspicious wallet patterns through unsupervised clustering of behavioral features.

This notebook follows the mission5/mission7 methodology:
1. **Data Loading**: Extract wallet transfer data from PostgreSQL
2. **Feature Engineering**: Build behavioral features for each wallet
3. **PCA Analysis**: Reduce dimensionality and visualize structure
4. **Clustering**: K-Means, DBSCAN with elbow/silhouette evaluation
5. **Profiling**: Interpret cluster characteristics
6. **MLflow**: Register best model and promote to champion

## 1. Setup & Imports

In [ ]:
# Standard libraries
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Add src to path for imports
sys.path.insert(0, os.path.abspath('../src'))

# Data
import numpy as np
import pandas as pd

# Visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = 'plotly_white'

# ML
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

# Custom classes
from classes.db_connector import DatabaseConnector
from classes.wallet_features import WalletFeatureExtractor
from classes.pca_analysis import PCAAnalysis
from classes.cluster_analysis import ClusterAnalysis
from classes.visualizer import WalletVisualizer
from classes.model_registry import ModelRegistry

print("✅ All imports successful")

## 2. Database Connection & Data Loading

In [ ]:
# Configuration - update these for your environment
DB_CONFIG = {
    'host': os.getenv('POSTGRES_HOST', 'localhost'),
    'port': os.getenv('POSTGRES_PORT', '5432'),
    'database': os.getenv('POSTGRES_DB', 'bubble'),
    'user': os.getenv('POSTGRES_USER', 'bubble'),
    'password': os.getenv('POSTGRES_PASSWORD', 'bubble_password')
}

MLFLOW_URI = os.getenv('MLFLOW_TRACKING_URI', 'http://localhost:5000')

print(f"📦 Database: {DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}")
print(f"📊 MLflow: {MLFLOW_URI}")

In [ ]:
# Connect to database
db = DatabaseConnector(
    host=DB_CONFIG['host'],
    port=DB_CONFIG['port'],
    database=DB_CONFIG['database'],
    user=DB_CONFIG['user'],
    password=DB_CONFIG['password']
)

# Show summary
db.summary()

In [ ]:
# Load transfer data
transfers_df = db.get_transfer_data(limit=1000000)
print(f"📥 Loaded {len(transfers_df):,} transfers")
transfers_df.head()

In [ ]:
# Load existing wallet labels (if any)
labels_df = db.get_wallet_labels()
print(f"🏷️ Found {len(labels_df):,} labeled wallets")
if len(labels_df) > 0:
    print(labels_df['category'].value_counts())

## 3. Feature Engineering

In [ ]:
# Extract wallet features
feature_extractor = WalletFeatureExtractor(transfers_df)
wallet_features = feature_extractor.extract_all_features()

print(f"\n📊 Extracted features for {len(wallet_features):,} wallets")
print(f"   Features: {len(wallet_features.columns)}")
wallet_features.head()

In [ ]:
# Normalize features
features_normalized = feature_extractor.normalize_features(wallet_features)
print("✅ Features normalized")
features_normalized.describe()

In [ ]:
# Visualize feature distributions
feature_cols = [c for c in features_normalized.columns if c != 'wallet_address']
fig = WalletVisualizer.plot_feature_distributions(
    features_normalized, 
    feature_cols[:9],  # First 9 features
    n_cols=3
)
fig.show()

In [ ]:
# Correlation matrix
fig = WalletVisualizer.plot_correlation_matrix(features_normalized, feature_cols)
fig.show()

## 4. PCA Analysis

In [ ]:
# PCA Analysis
pca_analyzer = PCAAnalysis(features_normalized[feature_cols], n_components=10)

# Explained variance
fig = pca_analyzer.plot_explained_variance()
fig.show()

In [ ]:
# PCA loadings heatmap
fig = pca_analyzer.plot_loadings_heatmap(n_components=6)
fig.show()

In [ ]:
# 2D scatter plot (PC1 vs PC2)
fig = pca_analyzer.plot_biplot(sample=5000)
fig.show()

In [ ]:
# 3D scatter plot
fig = pca_analyzer.plot_3d_scatter(sample=5000)
fig.show()

## 5. Clustering Analysis

In [ ]:
# Initialize cluster analyzer
cluster_analyzer = ClusterAnalysis(
    features_normalized, 
    features=feature_cols,
    n_pca_components=10
)

### 5.1 Elbow Method & Optimal K

In [ ]:
# Elbow analysis to find optimal K
elbow_results = cluster_analyzer.elbow_analysis(k_range=range(2, 15))

In [ ]:
# Plot elbow curves
fig = cluster_analyzer.plot_elbow()
fig.show()

In [ ]:
# Find best K by silhouette
best_k = elbow_results['k_range'][np.argmax(elbow_results['silhouette'])]
best_silhouette = max(elbow_results['silhouette'])
print(f"🎯 Optimal K: {best_k} (silhouette: {best_silhouette:.3f})")

### 5.2 K-Means Clustering

In [ ]:
# Fit K-Means with optimal K
cluster_analyzer.fit_kmeans(n_clusters=best_k)

In [ ]:
# Plot clusters in 2D
fig = cluster_analyzer.plot_clusters_2d(pc_x=1, pc_y=2, sample=5000)
fig.show()

In [ ]:
# Plot clusters in 3D
fig = cluster_analyzer.plot_clusters_3d(sample=5000)
fig.show()

In [ ]:
# Silhouette plot
fig = cluster_analyzer.plot_silhouette()
fig.show()

In [ ]:
# Cluster distribution
fig = WalletVisualizer.plot_cluster_distribution(cluster_analyzer.labels_)
fig.show()

### 5.3 DBSCAN Comparison

In [ ]:
# Estimate DBSCAN eps parameter
fig = cluster_analyzer.estimate_dbscan_eps(k=5)
fig.show()

In [ ]:
# Create a separate analyzer for DBSCAN comparison
dbscan_analyzer = ClusterAnalysis(
    features_normalized, 
    features=feature_cols,
    n_pca_components=10
)

# Fit DBSCAN (adjust eps based on k-distance plot)
dbscan_analyzer.fit_dbscan(eps=0.5, min_samples=5)

In [ ]:
# Compare metrics
print("\n📊 Clustering Comparison:")
print(f"   K-Means (K={best_k}):  silhouette = {cluster_analyzer.silhouette_:.3f}")
print(f"   DBSCAN:                silhouette = {dbscan_analyzer.silhouette_:.3f}")

# Select best method
if cluster_analyzer.silhouette_ >= dbscan_analyzer.silhouette_:
    best_analyzer = cluster_analyzer
    best_method = 'kmeans'
else:
    best_analyzer = dbscan_analyzer
    best_method = 'dbscan'

print(f"\n🏆 Best method: {best_method.upper()}")

## 6. Cluster Profiling

In [ ]:
# Get cluster profiles
profiles = best_analyzer.get_cluster_profiles()
print("📊 Cluster Profiles:")
profiles

In [ ]:
# Radar chart for each cluster
for cluster in range(best_analyzer.n_clusters_):
    fig = best_analyzer.plot_cluster_radar(cluster)
    fig.show()

In [ ]:
# Compare clusters
fig = WalletVisualizer.plot_wallet_comparison(
    profiles[feature_cols],
    features=feature_cols[:10],  # Top 10 features
    wallets=list(range(min(4, best_analyzer.n_clusters_)))  # First 4 clusters
)
fig.show()

In [ ]:
# Add cluster labels to original data
wallet_features_with_clusters = wallet_features.copy()
wallet_features_with_clusters['cluster'] = best_analyzer.labels_

# Save for later analysis
wallet_features_with_clusters.to_csv('../data/wallet_clusters.csv', index=False)
print("✅ Saved wallet clusters to ../data/wallet_clusters.csv")

## 7. MLflow Model Registry

In [ ]:
# Connect to MLflow
registry = ModelRegistry(
    tracking_uri=MLFLOW_URI,
    experiment_name='bubble_wallet_clustering'
)

In [ ]:
# Get metrics from best model
metrics = best_analyzer.get_metrics()
print("📊 Model Metrics:")
for k, v in metrics.items():
    print(f"   {k}: {v}")

In [ ]:
# Prepare parameters
params = {
    'method': best_method,
    'n_clusters': best_analyzer.n_clusters_,
    'n_features': len(feature_cols),
    'n_samples': len(features_normalized),
    'n_pca_components': best_analyzer.n_components
}

if best_method == 'kmeans':
    params['n_init'] = 10
elif best_method == 'dbscan':
    params['eps'] = 0.5
    params['min_samples'] = 5

In [ ]:
# Log run to MLflow
run_id = registry.log_clustering_run(
    cluster_model=best_analyzer.model_,
    metrics=metrics,
    params=params,
    features_df=features_normalized[feature_cols],
    run_name=f'clustering_{best_method}_{best_analyzer.n_clusters_}clusters',
    tags={'source': 'jupyter_notebook', 'version': '1.0'}
)

In [ ]:
# Register model
MODEL_NAME = 'bubble_cluster_model'
version = registry.register_model(run_id, model_name=MODEL_NAME)

In [ ]:
# Compare with current champion
comparison = registry.compare_with_champion(
    new_metrics=metrics,
    model_name=MODEL_NAME,
    metric_name='silhouette'
)

In [ ]:
# Promote to champion if better
if comparison['is_better']:
    print("\n🏆 New model is better! Promoting to Champion...")
    success = registry.promote_to_champion(
        model_name=MODEL_NAME,
        version=version,
        metric_name='silhouette'
    )
    if success:
        print(f"✅ Model v{version} is now the Champion!")
else:
    print("\n❌ Current champion is still better. Not promoting.")

In [ ]:
# Show experiment history
history = registry.get_experiment_runs(max_results=10)
history[['run_name', 'silhouette', 'n_clusters', 'method', 'start_time']]

## 8. Summary & Next Steps

In [ ]:
print("\n" + "="*60)
print("📋 CLUSTERING ANALYSIS SUMMARY")
print("="*60)
print(f"\n📦 Data:")
print(f"   - Transfers analyzed: {len(transfers_df):,}")
print(f"   - Unique wallets: {len(wallet_features):,}")
print(f"   - Features extracted: {len(feature_cols)}")

print(f"\n📊 Clustering:")
print(f"   - Best method: {best_method.upper()}")
print(f"   - Number of clusters: {best_analyzer.n_clusters_}")
print(f"   - Silhouette score: {best_analyzer.silhouette_:.3f}")

print(f"\n📈 MLflow:")
print(f"   - Run ID: {run_id}")
print(f"   - Model version: {version}")
print(f"   - Is champion: {comparison['is_better']}")

print(f"\n📁 Outputs:")
print(f"   - Cluster labels: ../data/wallet_clusters.csv")
print(f"   - MLflow UI: {MLFLOW_URI}")

print("\n" + "="*60)

In [ ]:
# Interpretation suggestions
print("\n🔍 Cluster Interpretation Suggestions:")
print("\nBased on typical wallet behaviors:")
interpretations = {
    'High tx_count + High unique_counterparties': 'Exchange/Hub wallet',
    'High in_out_ratio (>2) + Low diversity': 'Aggregation/Mixer pattern',
    'High burst_ratio + Short activity_span': 'Bot/Automated trading',
    'Low tx_count + High avg_value': 'Whale/High-value holder',
    'Outlier in PCA space': 'Potentially suspicious behavior'
}

for pattern, interpretation in interpretations.items():
    print(f"   • {pattern}")
    print(f"     → {interpretation}")

---
**Next Steps:**
1. Review cluster profiles and assign semantic labels (Exchange, Bot, Mixer, Normal, etc.)
2. Use cluster labels to train a supervised classifier
3. Integrate with the Investigation workflow in the main Bubble application
4. Set up scheduled re-clustering as new data arrives